<a href="https://colab.research.google.com/github/Sofanit/HandVideo/blob/main/Uned47.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
'''criticality as part of pypullet'''
!pip install stable-baselines3[extra] pybullet

In [12]:
import numpy as np
if not hasattr(np, 'bool8'):
    np.bool8 = np.bool_


In [13]:
import gym
import pybullet_envs
from stable_baselines3 import SAC
from stable_baselines3.common.env_util import make_vec_env

In [18]:
from stable_baselines3.common.env_util import make_vec_env
from gym.wrappers import StepAPICompatibility

# Old gym env from pybullet
raw_env = gym.make("ReacherBulletEnv-v0")

# Wrap for compatibility with new Gym API
wrapped_env = StepAPICompatibility(raw_env, new_step_api=False)
env = make_vec_env(lambda: wrapped_env, n_envs=1)

In [ ]:
model = SAC("MlpPolicy", env, verbose=1)
model.learn(total_timesteps=10000)

In [ ]:
def collect_states(model, env, steps=500):
    obs = env.reset()
    if isinstance(obs, np.ndarray):
        obs = obs.astype(np.float32)
    states = [obs.copy()]

    for _ in range(steps):
        action, _ = model.predict(obs, deterministic=True)
        obs, reward, done, info = env.step(action)
        if isinstance(obs, np.ndarray):
            obs = obs.astype(np.float32)
        states.append(obs.copy())
        if done:
            obs = env.reset()
    return np.array(states)

In [40]:
def collect_states(model, env, steps=500):
    obs = env.reset()
    # Unwrap if using VecEnv
    if isinstance(obs, tuple): obs = obs[0]
    if isinstance(obs, np.ndarray): obs = obs.astype(np.float32)

    states = [obs.copy()]

    for _ in range(steps):
        action, _ = model.predict(obs, deterministic=True)
        obs, reward, done, info = env.step(action)
        if isinstance(obs, tuple): obs = obs[0]
        if isinstance(obs, np.ndarray): obs = obs.astype(np.float32)
        states.append(obs.copy())

        if done:
            obs = env.reset()
            if isinstance(obs, tuple): obs = obs[0]

    return np.array(states)  # shape = (steps+1, obs_dim)

In [33]:
def compute_criticality(states):
    return np.linalg.norm(states[1:] - states[:-1], axis=1)


In [53]:

states = collect_states(model, env, steps=10000)
print("states shape:", states.shape)  #

states shape: (10001, 1, 9)


In [54]:
crit_scores = compute_criticality(states)
print("crit_scores shape:", crit_scores.shape)

crit_scores shape: (10000, 9)


In [ ]:
env_eval = gym.make("ReacherBulletEnv-v0")
obs = env_eval.reset()
total_reward = 0

for _ in range(200):
    action, _ = model.predict(obs, deterministic=True)
    obs, reward, done, info = env_eval.step(action)
    total_reward += reward
    if done:
        break

print("Total episode reward:", total_reward)

In [55]:
import pickle
with open("criticality_scores_2000.pkl", "wb") as f:
    pickle.dump(crit_scores, f)